In [129]:
import os
import sys
import requests
import re
import time
import json
import pandas as pd
import csv
from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

In [130]:
extracts = re.compile('[^ 가-힣|a-z|A-Z|0-9|\[|\]|(|)|-|~|?|!|.|,|:|;|%]+')
cost_extracts = re.compile('[^ 가-힣|a-z|A-Z|\[|\]|(|)|-|~|?|!|.|,|:|;|%]+')

In [131]:
# 서울
# data_url = open('./data/goodchoice_seoul.csv', 'w', encoding='utf8') # url 저장 관련 csv파일
# data = open('./data/info_seoul.csv', 'w', encoding='utf8') # 정보 저장관련 csv 파일

# 부산
data_url = open('./data/goodchoice_busan.csv', 'w', encoding='utf8') # url 저장 관련 csv파일
data = open('./data/info_busan.csv', 'w', encoding='utf8') # 정보 저장관련 csv 파일
data_r = open('./data/reviews_busan.csv', 'w', encoding='utf8') # 리뷰 저장관련 csv 파일

In [132]:
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window() #전체 화면
# Url = 'https://www.goodchoice.kr/product/search/1/7052' #서울 역삼, 강남, 논현
Url = 'https://www.goodchoice.kr/product/search/1/7044' #부산 광안리, 수영, 민락
driver.get(Url)

In [133]:
urls = driver.find_element(By.ID, "poduct_list_area")
urlList = urls.find_elements(By.TAG_NAME, 'li')

for li in urlList:
    aTag = li.find_element(By.TAG_NAME, 'a')
    href = aTag.get_attribute('href')
    data_url.write(href)
    data_url.write('\n')

data_url.close()    

In [134]:
data.write('"name"')
data.write(', ')
data.write('"basic_info"')
data.write(', ')
data.write('"cost_info"')
data.write(', ')
data.write('"avg_stars"')
data.write('\n')

1

In [135]:
data_r.write('"stars"')
data_r.write(', ')
data_r.write('"review"')
data_r.write('\n')

1

In [136]:
# 서울
# f = open('C:/Users/xmssn/OneDrive/바탕 화면/대학/3학년1학기/빅데이터시스템/project/python_01/data/goodchoice_seoul.csv', 'r', encoding='utf8')

# 부산
f = open('C:/Users/xmssn/OneDrive/바탕 화면/대학/3학년1학기/빅데이터시스템/project/python_01/data/goodchoice_busan.csv', 'r', encoding='utf8')
rdr = csv.reader(f)

In [137]:
# 기본 정보, 요금 정보 및 평점
for line in rdr:
    addr = line[0]
    rs = requests.get(addr)
    rs.encoding = None
    html = rs.text
    soup = bs(html, 'html.parser')

    try:
        tarname = soup.select('#content > div.top > div.right')
        rname = soup.find("div", {"class" : "info"})
        name = rname.find("h2")
        name = name.text

        tarbasic_info = soup.select('#content > article.detail_info > section.default_info')
        basic_info = ''
        for text in tarbasic_info:
            basic_info += text.getText()

        basic_info = extracts.sub('', basic_info)
        basic_info = re.sub(' +', ' ', basic_info)
        basic_info = '"'+basic_info+'"'


        tarcost_info = soup.select('#content > article.detail_info.on > section.table_wrap > div:nth-child(2) > table:nth-child(2)')
        rcost_info = soup.find("table", {"class" : "table_type_02"})
        cost_info = rcost_info.find("span")
        cost_info = cost_info.text


        tarstar_info = soup.select('#content > div.top > div.right > div.info')
        rstar_info = soup.find("div", {"class" : "score_cnt"})
        avg_star_info = rstar_info.find("span").text[0:4]

        data.write(name)
        data.write(', ')
        data.write(basic_info)
        data.write(', ')
        data.write(cost_info)
        data.write(', ')
        data.write(avg_star_info)
        data.write('\n')
    except:
        continue

data.close()

In [138]:
# for line in rdr:
#     addr = line[0]
#     rs = requests.get(addr)
#     rs.encoding = None
#     html2 = rs.text
#     soup2 = bs(html2, 'html.parser')

#     try:
#         tarstar = soup2.select('#review > ul > li:nth-child(1) > div')
#         rstar = soup2.find("div", {"class" : "score_wrap_sm"})
#         star = rstar.find("num").text[0:4]

#         data_r.write(star)
#         data_r.write('\n')
#     except:
#         break

# f.close()
# data_r.close()

In [139]:
# for page in range(1, 5):
#     review_addr = addr+"#{}".format(page)
#     html2 = requests.get(review_addr)
#     html2.encoding = None
#     html2 = html2.text
#     soup2 = bs(html2, 'html.parser')

#     for i in range(1, 11):
#         tarstar = soup2.select("#review > ul > li:nth-child({}) > div".format(i))
#         rstar = soup2.find("div", {"class" : "score_wrap_sm"})
#         star = rstar.find("div", {"class":"num"}).text[0:4]
#         data_r.write(star)
#         data_r.write('\n')


    

# f.close()
# data_r.close()
        